# Massage Sharma2024 Table

In [2]:
# imports
import numpy as np
import pandas

from frb import utils as frb_utils

# Load up

In [3]:
df = pandas.read_csv('sharma2024_orig.csv')

In [4]:
df.head()

,frb_name,sample,P_host,sed_done?,FRB_TNS_name,instrument,"host_loc (deg, deg)","host_loc (hms, dms)",redshift,redshift_err,...,plot2_prospector?,plot3_prospector?,w1-w2,w1-w2_err,w2-w3,w2-w3_err,r_mag,r_err,m_r or m_J,m_r_err or m_J_err
0,mark,Gold,PS1/r 0.999,Y,FRB 20220319D,P-200/DBSP,"32.170259020, +71.035874990","02:08:40.86, 71:02:09.150",0.0112,0.0000,...,Y,Y,0.10,0.03,3.37,0.03,13.25,0.01,13.25,0.01
1,mizu,Gold,BASS/r 0.987,Y,FRB 20231120A,Keck/LRIS,"143.984540350, 73.284945000","09:35:56.29, 73:17:05.80",0.0368,0.0000,...,N,N,0.11,0.03,3.63,0.03,15.39,0.01,15.41,0.21
2,zach,Gold,PS1/r 0.969,Y,FRB 20220207C,Keck/LRIS,"310.197579, +72.882749","20:40:47.4189, +72:52:57.897",0.0433,0.0001,...,Y,Y,0.15,0.05,3.65,0.07,16.15,0.01,16.15,0.01
3,jackie,Gold,BASS/r 0.969,Y,FRB 20220509G,Keck/LRIS,"282.674784770, +70.242733870","18:50:41.9158, +70:14:33.951",0.0894,0.0000,...,Y,Y,0.13,0.03,2.89,0.05,16.51,0.01,16.51,0.21
4,leonidas,Gold,BASS/r 0.999,Y,FRB 20230124A,Keck/DEIMOS,"231.915836640, +70.968226560","15:27:39.800, +70:58:05.620",0.0939,0.0002,...,N,N,0.12,0.06,3.43,0.17,19.18,0.03,19.10,0.20


In [5]:
df.iloc[6]['host_loc (deg, deg)']

'166.786333, 72.281392'

# Add RA, Dec

In [6]:
float(df.iloc[0]['host_loc (deg, deg)'].split(',')[0])

32.17025902

In [7]:
ra = [float(item.split(',')[0]) for item in df['host_loc (deg, deg)'].values]
ra[0:5]

[32.17025902, 143.98454035, 310.197579, 282.67478477, 231.91583664]

In [8]:
dec = [float(item.split(',')[1]) for item in df['host_loc (deg, deg)'].values]

In [9]:
df['ra'] = ra
df['dec'] = dec

# Rename according the galaxy defs

## https://github.com/FRBs/FRB/blob/main/frb/galaxies/defs.py

In [10]:
rdict = dict(logMstar='Mtotal', logMstar_errl='Mtotal_loerr', logMstar_erru='Mtotal_uperr',
             #ssfr='
             #logzsol
             FRB_TNS_name='Name', 
             sfr_100Myr='SFR_SED', sfr_100Myr_errl='SFR_SED_loerr',  sfr_100Myr_erru='SFR_SED_uperr',
             Av_old='AV_old',  Av_old_errl='AV_old_loerr',  Av_old_erru='AV_old_uperr', 
            )
# Nebular lines
#rdict['[SII]6716'] = '[SII] 6716'
#rdict['[SII]6716_err'] = '[SII] 6716_err'
#rdict['[OIII]5007_d'] = '[OIII] 5007'
#rdict['[OIII]5007_d_err'] = '[OIII] 5007_err'

## And deal with a few log/linear issues

In [11]:
# Rename
df_new = df.rename(columns=rdict)
#
# Log/linar
for key in ['Mtotal']:
    df_new[key] = 10**df_new[key]
    # Errors
    key_l, key_u = key+'_loerr', key+'_uperr'
    lo_err, up_err = [], []
    for val, up, lo in zip(df_new[key].values, df_new[key_u].values, df_new[key_l].values):
        lin_up, lin_lo = frb_utils.log10_to_linear_errors(val, up, lo)
        lo_err.append(lin_lo)
        up_err.append(lin_up)
    df_new[key_l] = lo_err 
    df_new[key_u] = up_err 

In [12]:
hgnames = [item.replace('FRB','HG') for item in df_new.Name]
df_new['Name'] = hgnames

# Cut down to good columns

In [13]:
gd_col = ['ra', 'dec']#, 'Halpha', 'Halpha_err', 'Hbeta', 'Hbeta_err']
for key in rdict.keys():
    gd_col.append(rdict[key])
#
df_new = df_new[gd_col]
#gd_col

# Write

In [14]:
df_new.to_csv('sharma2024_derived.csv', index=False)

In [15]:
df_new.keys()

Index(['ra', 'dec', 'Mtotal', 'Mtotal_loerr', 'Mtotal_uperr', 'Name',
       'SFR_SED', 'SFR_SED_loerr', 'SFR_SED_uperr', 'AV_old', 'AV_old_loerr',
       'AV_old_uperr'],
      dtype='object')

In [16]:
df_new.head()

,ra,dec,Mtotal,Mtotal_loerr,Mtotal_uperr,Name,SFR_SED,SFR_SED_loerr,SFR_SED_uperr,AV_old,AV_old_loerr,AV_old_uperr
0,32.170259,71.035875,1.260956e+10,-1.225368e+08,6.403816e+07,HG 20220319D,0.4238,-0.0209,0.0222,0.3400,-0.0100,0.0100
1,143.984540,73.284945,2.506686e+10,-1.215026e+08,1.273031e+08,HG 20231120A,0.4019,-0.0250,0.0189,0.7162,-0.0068,0.0068
2,310.197579,72.882749,8.873603e+09,-7.027467e+08,6.477791e+08,HG 20220207C,0.7124,-0.1347,0.1244,0.0956,-0.0181,0.0233
3,282.674785,70.242734,5.021113e+10,-1.075008e+09,1.501901e+09,HG 20220509G,0.2542,-0.0396,0.0661,0.0496,-0.0155,0.0184
4,231.915837,70.968227,2.890680e+09,-1.869711e+07,2.339025e+07,HG 20230124A,0.7524,-0.0381,0.0406,0.0280,-0.0050,0.0043


In [19]:
a = ','.join([item.replace(' ','') for item in df.FRB_TNS_name])
a, len(a.split(','))

('FRB20220319D,FRB20231120A,FRB20220207C,FRB20220509G,FRB20230124A,FRB20220914A,FRB20230628A,FRB20220920A,FRB20221101B,FRB20220825A,FRB20220307B,FRB20221113A,FRB20231123B,FRB20230307A,FRB20221116A,FRB20221012A,FRB20220506D,FRB20230501A,FRB20230626A,FRB20220208A,FRB20220726A,FRB20220330D,FRB20220204A,FRB20230712A,FRB20220310F,FRB20230216A,FRB20221027A,FRB20221219A,FRB20220418A,FRB20221029A',
 30)

----

In [19]:
dfg = pandas.read_csv('gordon2023_derived.csv')
np.array(dfg.keys())

array(['Name', 'ra', 'dec', 'z_SED', 'z_SED_uperr', 'z_SED_loerr',
       'Mtotal', 'Mtotal_uperr', 'Mtotal_loerr', 'Z_stellar',
       'Z_stellar_uperr', 'Z_stellar_loerr', 'AV_young', 'AV_young_uperr',
       'AV_young_loerr', 'AV_old', 'AV_old_uperr', 'AV_old_loerr',
       'f_AGN', 'f_AGN_uperr', 'f_AGN_loerr', 'agn_tau', 'agn_tau_uperr',
       'agn_tau_loerr', 'Z_gas', 'Z_gas_uperr', 'Z_gas_loerr', 'SFR_SED',
       'SFR_SED_uperr', 'SFR_SED_loerr', 'Mstar', 'Mstar_uperr',
       'Mstar_loerr', 'age_mass', 'age_mass_uperr', 'age_mass_loerr',
       'lg_sSFR', 'lg_sSFR_uperr', 'lg_sSFR_loerr'], dtype=object)